# AgentX Quickstart

Welcome to the XEntropy quickstart guide, your first step in exploring the world of application development with agent-based large language models (or in short, agentic llm).

XEntropy is an innovative development platform designed to elevate the ease at which developers can construct intelligent applications. Throughout this tutorial, we will navigate the following key areas:

1. **Account creation:** We guide you step-by-step in establishing your XEntropy account:the gateway to a host of agentic llm tools.

2. **Loading tools from XEntropy:** Learn how to seamlessly and efficiently import tools from the platform into your environment to kickstart your foray into developing llm agents and intelligent applications.

3. **Managing your account:** Your XEntropy account is useful for many things. Learn how to:
    - **Purchase credit**: Convenience and accessibility are core to XEntropy. Understand how to replenish your account balance in a straightforward, user-friendly manner.

   - **Get paid**: Having created applications or tools that other developers find useful? Here's how to benefit from your creations.

   - **Modify and delete tools**: As your skill level advances, so might your tools and applications. Learn how to adjust and remove tools from your portfolio.

   - **Utilisation of logs**: Track your progress and rectify mistakes by understanding how to make use of logs effectively.

   - **Finetune your own agentic llm**: Capture the essence of your application development ideation by discovering how to fine-tune your personal agentic llm to reach peak performance.

## Installation

In [1]:
# Install AgentX
from distutils.dir_util import copy_tree, remove_tree
import os
import sys

site_packages_path = [x for x in sys.path if x.endswith('site-packages')][0]
# remove any existing installation
if os.path.exists(f'{site_packages_path}/agentx'):
    remove_tree(f'{site_packages_path}/agentx')
copy_tree(f'../agentx', f'{site_packages_path}/agentx')

['C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\agent.py',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\bedrock_client.py',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\client.py',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\groupchat.py',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\models\\.gitignore',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\models\\__init__.py',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\models\\__pycache__\\__init__.cpython-310.pyc',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\oai_client.py',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\optimisers.py',
 'C:\\Users\\jerem\\AppData\\Roaming\\Python\\Python310\\site-packages/agentx\\saved_agents.py',
 'C:\\Users

In [2]:
# Install dependencies to complete this tutorial
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Create your account

In [3]:
from agentx.client import Client

In [4]:
# load environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from agentx.tool import Tool
import os

tool_search = Tool.load(
    'xentropy--tool_search', 
    api_key=os.environ.get('XENTROPY_API_KEY')
)

# display the function call schema
tool_search.input_json_schema

{'title': 'SearchQuery',
 'type': 'object',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query']}

In [6]:
from rich import print as rich_print
import json
# run the tool
rich_print(
    json.loads(
        tool_search.run(query='convert an address to geolocation')
    )
)

[
    {
        'tool_id': 'xentropy--geocoding',
        'description': 'Retrieve the latitude and longitude given an address using the highly accurate Google Map 
API.',
        'score': 0.85867697
    },
    {
        'tool_id': 'xentropy--geodesic',
        'description': 'Calculate the earth surface distance between two latitude and longitude coordinate',
        'score': 0.792058647
    },
    {
        'tool_id': 'xentropy--restaurant_search',
        'description': 'Search restaurants based on food preference, location, and other user requirements. 
Examples: {"include_type": "korean_restaurant", "query": "Korean barbeque in Mountain View with spacious seating 
and good service"}',
        'score': 0.719365895
    },
    {
        'tool_id': 'xentropy--tool_search',
        'description': 'Search for relevant tools to accomplish a task.',
        'score': 0.686780155
    },
    {
        'tool_id': 'xentropy--etf_search',
        'description': 'Search for ETFs that satisfy a given description.',
        'score': 0.674237192
    },
    {
        'tool_id': 'xentropy--question_answering_by_product_brochure',
        'description': 'Base on an insurance product brochure answer the query page by page.',
        'score': 0.668316543
    }
]

In [7]:
# Get a tool that converts address to latitude longitude coordinate
geocoding = Tool.load('xentropy--geocoding', api_key=os.environ.get('XENTROPY_API_KEY'))
# Get a tool that computes the earth surface distance between two coordinates
geodesic = Tool.load('xentropy--geodesic', api_key=os.environ.get('XENTROPY_API_KEY'))

## Creating Agents with the loaded tools

In [22]:
from agentx.agent import Agent
from agentx.schema import GenerationConfig, Message, Content
from typing import List

tools = [geocoding, geodesic]

#generation_config = GenerationConfig(
#    api_type='vertexai',
#    path_to_google_service_account_json='vertex_ai_creds.json',
#    google_application_credential_scope=['https://www.googleapis.com/auth/cloud-platform'],
#    region='asia-northeast3',
#)

generation_config = GenerationConfig(
    api_type='azure',
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    api_version="2024-02-15-preview",
    base_url="https://genai-instance-test-jeremy.openai.azure.com/", 
)

# replace with your own azure deployment to choose different models
#generation_config.model = 'gemini-pro'
generation_config.azure_deployment = 'gpt-35-jeremy'

print(generation_config)

# define a termination function
# once the agent execute the geodesic tool, the conversation ends
def terminate(history:List[Message]):
    tool_response_name = [
        message.content.tool_response.name for message in history if message.content.tool_response
    ]
    if 'xentropy--geodesic' in tool_response_name:
        return True
    return False
    
# define the agent
agent = Agent(
    name='agent',
    generation_config=generation_config,
    system_prompt=None,#'Use the functions you have been provided to solve the problem. Reply TERMINATE to end the conversation when the problem is solved.',
    tools = tools,
    termination_function=terminate,
)

api_type='azure' api_key='a5a4a343ef6f4897a9e7883ac90ca3d8' api_version='2024-02-15-preview' organization=None base_url=Url('https://genai-instance-test-jeremy.openai.azure.com/') timeout=120 max_retries=3 azure_deployment='gpt-35-jeremy' path_to_google_service_account_json=None google_application_credential_scope=None region=None project_id=None model=None n_candidates=1 frequency_penalty=None logit_bias=None max_tokens=None presence_penalty=None response_format=None seed=None stop_sequences=None temperature=None tool_choice=None tools=None top_p=None top_k=None
{'xentropy--geocoding': Function(name='xentropy--geocoding', description='Retrieve the latitude and longitude given an address using the highly accurate Google Map API.', parameters={'title': 'Address', 'type': 'object', 'properties': {'address': {'title': 'Address', 'type': 'string'}}, 'required': ['address']}), 'xentropy--geodesic': Function(name='xentropy--geodesic', description='Calculate the earth surface distance between

In [23]:
# the agent is ready to generate messages
messages = [
    Message(
        role='user',
        content=Content(
            text='What is the distance between Gare Port La Goulette - Sud in Tunisia and Porto di Napoli in Italy?',
        ),
    )
]

In [24]:
# use the agent to use geocoding and geodesic to find earth surface distance between two locations

max_iterations = 10

for i in range(max_iterations):
    response = agent.generate_response(messages)
    # termination condition is met
    print(response)
    if response == None:
        break
    else:
        [rich_print(r.model_dump(exclude_unset=True)) for r in response]
    messages += response

NotFoundError: Error code: 404 - {'error': {'message': 'Unrecognized request argument supplied: tools', 'type': 'invalid_request_error', 'param': None, 'code': None}}

### Share and get paid for your tools
You can checkout the following examples to see how to create and share tools on XEntropy.
- [api](https://github.com/ckh112/xentropy/blob/main/docs/api-tool/notebook.ipynb)
- [agent](https://github.com/ckh112/xentropy/blob/main/docs/composite-agents-tool/notebook.ipynb)
- [rag](https://github.com/ckh112/xentropy/blob/main/docs/rag-tool/notebook.ipynb)

## Managing your account

The `Client` class has a few other utilities to access XEntropy platform.

In [8]:
from agentx.client import Client
client = Client(api_key=os.environ.get('XENTROPY_API_KEY'))

# See your account summary and usage data on each tool
client.summary()

In [ ]:
# XEntropy credit can be used to pay for tool usage, if the tool is not free.
# You can top up your XEntropy credit by paying with credit card by navigating to the payment link.
print(client.summary().get('payment_link'))

In [ ]:
# Alternatively you can also top up your XEntropy credit by paying with cryptocurrency.
# We accept USDT, USDC, and DAI on the ethereum blockchain.
# First, register your address.
from eth_account import Account
account = Account.from_key('YOUR_PRIVATE_KEY')
# you can also use mnemonic phrase
# account = Account.from_mnemonic('YOUR_MNEMONIC_PHRASE')
response = client.register_ethereum_address(account=account)
print(response.json())

In [ ]:
# Sometimes you'd like to modify your published tool
# Example 1: Making the tool public to XEntropy platform user
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="public",
    value=True,
)

In [ ]:
# Example 2: Tool description acts as a prompt to LLM, and hence it can be optimised through prompt engineering.
# You may want to modify it for improved performance.
# Check out https://github.com/microsoft/LMOps/tree/main/prompt_optimization for inspiration.
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="description",
    value="YOU_NEW_DESCRIPTION",
)

In [ ]:
# You can take your tool away anytime.
client.delete_tool(
    tool="YOUR_TOOL_NAME"
)

XEntropy pays 80% of the tool-use revenue to tool developers's `payout` wallet. There is also a 5% bonus paid to the `balance` wallet to incentivize tool developers for consuming other tools on XEntropy.

At this moment we only support withdrawl on the ethereum network for selected stable coins. More withdrawal method is on the way.

In [ ]:
# To withdraw from your payout wallet
client.stable_coin_payout(
    amount=100000,  # amount to withdraw denominated in XEntropy Credit. i.e. 100000 XEntropy Credit = 1 USD
    address="YOUR_ETHEREUM_ADDRESS", # your ethereum network address
    stable_coin='USDT' # choose between ['USDT', 'USDC', 'DAI']
)